In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import fastai
from fastai.text.all import *

In [2]:
df = pd.read_csv("../data/quests/quest_objectives.csv", sep=";").drop(columns="Unnamed: 0")

In [3]:
df.head()

,source,id,title,objectives
0,skyrim,55074,A Blade in the Dark,Talk to Delphine \nLocate the dragon burial site \nKill the Dragon Sahloknir \nReturn to Delphine
1,skyrim,55212,A Chance Arrangement,Talk to Brynjolf about joining the Thieves Guild \nMeet Brynjolf during daytime \nSteal Madesi's ring from his strongbox \nPlant Madesi's ring \nSpeak to Brynjolf
2,skyrim,51114,A Cornered Rat,Talk to Brynjolf \nSearch the Ratway for Esbern's hideout \nFind Esbern in the Ratway Warrens \nTalk to Esbern
3,skyrim,52017,A Daedra's Best Friend,Talk with Barbas \nTravel with Barbas to the shrine of Clavicus Vile \nJoin Barbas outside Haemar's Shame \nRetrieve the Rueful Axe with Barbas \nReturn to the shrine of Clavicus Vile with Barbas and the Rueful Axe \nGive the Rueful Axe to Clavicus Vile or kill Barbas with the Rueful Axe \nSpeak to Clavicus Vile
4,skyrim,63392,A False Front (Imperials),Find the Stormcloak courier \nRetrive the Stormcloak courier's package \nBring the documents to Legate Rikke \nBring the forged documents to Frorkmar Banner-Torn


In [49]:
df["text"] = df.apply(lambda x: (x["title"] + " > " + x["objectives"].replace("\n", " > ")).lower(), axis=1)

In [50]:
df.sample(3)["text"]

322                           under the table > deliver mead to wilhelm (option one)  > show mead to indaryn (option two)
465                                           veni vidi vigo > sneak onto the emperor's flagship under cover of darkness.
41     bring it! > speak to harrald about the dagger for the son of the jarl  > pick up his sword from balimund in riften
Name: text, dtype: object

---

In [51]:
dls = TextDataLoaders.from_df(df, text_col="text", is_lm=True, seed=66)

/home/tiago/anaconda3/envs/creative-ai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [63]:
learn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()], path="../models/", wd=0.1).to_fp16()

In [64]:
title = "A Blade in the Dark"

In [65]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > Attacked ( KNOT ) can be found on The Set Down of MIDNIGHT .'

In [66]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.724094,6.435636,0.163411,623.678833,00:00


In [67]:
learn.save('1epoch')

Path('../models/models/1epoch.pth')

In [68]:
learn = learn.load('1epoch')

In [69]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.605460,5.486403,0.195204,241.387375,00:01
1,5.852122,4.149859,0.292426,63.425083,00:01
2,5.232127,3.726447,0.339193,41.531269,00:01
3,4.815812,3.450250,0.372287,31.508272,00:01
4,4.511985,3.304797,0.392470,27.243002,00:01
5,4.273226,3.205766,0.402669,24.674387,00:01
6,4.088292,3.152278,0.415039,23.389296,00:01
7,3.940469,3.120916,0.415473,22.667141,00:01
8,3.817215,3.107440,0.417535,22.363731,00:01
9,3.719138,3.104773,0.418837,22.304146,00:01


In [70]:
learn.save_encoder('finetuned')

In [71]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > help the fiend activate the city of your witcher senses . sunken , take me snow 3 / 3'

In [75]:
title = "A magical sword"
learn.predict(title.lower() + ' > ', n_words=20)

'a magical sword > kill the iron sword > use your vigi senses to get all the elements killed diagram > return to footsteps'

In [76]:
title = "A magical sword"
learn.predict(title.lower() + ' > ', n_words=20)

'a magical sword > search for artifacts floor > report to tournament > steal a ring hraggstad off tower ( quest ) >'

In [72]:
title = "A curious looking egg"
learn.predict(title.lower() + ' > ', n_words=20)

"a xxunk xxunk xxunk > read a letter to grossbart > read the letter of chort > identify the haunted baron with clavicus 's letter"

---

In [55]:
learn.fine_tune(10, 1e-2)

epoch,train_loss,valid_loss,time
0,6.589756,5.739772,00:00


epoch,train_loss,valid_loss,time
0,5.706264,4.869517,00:01
1,5.222791,4.101242,00:01
2,4.813826,3.628067,00:01
3,4.486436,3.427560,00:01
4,4.238911,3.203441,00:01
5,4.023324,3.121645,00:01
6,3.852403,3.077629,00:01
7,3.714095,3.050021,00:01
8,3.595885,3.038499,00:01
9,3.501235,3.035754,00:01


In [57]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > talk to the black man > collect boots word from killing the baron of his home . > retrieve gold'

In [58]:
learn.predict(title.lower() + ' > ', n_words=20)

'a blade in the dark > bring power to lower giver the family battle > look for the guardian leader fort in sirens . >'

In [59]:
title = "The Crimson Conspiracy"

In [60]:
learn.predict(title.lower() + ' > ', n_words=20)

'the crimson xxunk > locate high and water vilkas notes to the 25 citizens of the ruins of woe against earth . >'

In [61]:
title = "A curious looking egg"

In [62]:
learn.predict(title.lower() + ' > ', n_words=20)

'a xxunk xxunk xxunk > find the ring on orchard of wood > find the way to the obelisk . > use the obelisk'